In [343]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math

In [344]:
# filename=r'/Users/Amelia/Desktop/DAL.csv'

In [345]:
# def makeDataFrame(filename):
#     data=pd.read_csv(filename)
#     df=pd.DataFrame(data,columns=['Adj Close'])
#     print(df)
#     s=df.values.tolist()
#     return s 

makeDataFrame(filename)

In [346]:
# parameters
S0=30
r=0.06
t=4
u=1+r+math.sqrt((1+r)**2-1) 
drift=0
vol=10 # for additive amount, how much to add
d=(1+r)/u
face=50
coupon_rate = 0.04
coupon = coupon_rate*face
default_threshold = 1/3
bankrupcy_ratio = 2.4
K=55 # call value

In [347]:
def g(x): # payoff function e.g for a call it would be max(0, x-K)
    if x==0:
        return [0, 0]
    if x <= default_threshold*S0:
        return [x*bankrupcy_ratio, 0]
    return [face, coupon] # first entry is the actual value, second is dividends or coupons

K2 has to be greater than K1

In [348]:
p=0.5#(1+r-d)/(u-d)
q=1-p
p=1-q # fixes floating-point errors hopefully
print(p, q) # only use these in the multiplicative model

0.5 0.5


In [349]:
def nSigFigs(num, n):
    #print(num == np.nan)
    if type(num) not in [float, int, np.float64] or num == float("NaN"): return float("NaN")
    return int(num * 10**n) / 10**n

In [350]:
def displayChart(tab):
    Chart=[]
    columns = []
    N = len(tab)-1
    for i in range(N, -1, -1):
        chart = []
        columns.append(N-i)
        for j in range(0,i):
            chart.append("")
        for j in range(i, N+1):
            entry = tab[j][i]
            chart.append(entry)
        Chart.append(chart)
    df = pd.DataFrame(Chart, columns = columns)
    return df.style.hide_index()

In [351]:
def r_values(drift, vol):
    R = {}
    for i in range(t, -1, -1):
        R[i] = r + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0, i+1, 1)
    return R

rates = r_values(0, 0.005)
print("interest rates")
displayChart(rates)

interest rates


0,1,2,3,4
,,,,0.080000
,,,0.075000,0.070000
,,0.070000,0.065000,0.060000
,0.065000,0.060000,0.055000,0.050000
0.060000,0.055000,0.050000,0.045000,0.040000


In [352]:
def StockPrice(mode='a'): # a for additive, m for multiplicative
    StockPrice={}
    if mode == 'm':
        for i in range(t, -1, -1):
            StockPrice[i] = S0 * d ** (np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #np.empty(N+1)

    elif mode == 'a':
        for i in range(t, -1, -1):
            StockPrice[i] = np.maximum(np.zeros(i+1), S0 + i*drift - vol*np.arange(i, -1, -1) + vol*np.arange(0,i+1,1))
    '''for n in range(0, N+1):
        StockPrice[N].append(S0*(u**n)*(d**(N-n)))
    for i in range(N-1, -1, -1):
        StockPrice[i]=[] #np.empty(i+1)
        for j in range(0, i+1):
            StockPrice[i].append((StockPrice[i+1][j+1])/u)'''
    return StockPrice

SP = StockPrice()
print("stock price")
displayChart(SP)

# for i in range(t, -1, -1):
#     print(str(i) + ": "+str([nSigFigs(x, n=4) for x in SP[i]]))

stock price


0,1,2,3,4
,,,,70.000000
,,,60.000000,50.000000
,,50.000000,40.000000,30.000000
,40.000000,30.000000,20.000000,10.000000
30.000000,20.000000,10.000000,0.000000,0.000000


In [353]:
def p_values(stock_price_chart, rates):
    P = {i: [1]*(i+1) for i in range(t+1)}
    for i in range(0, t):
        for j in range(i+1):
            if stock_price_chart[i][j] == 0:
                P[i][j] = -1
            else:
                d = stock_price_chart[i+1][j] / stock_price_chart[i][j]
                u = stock_price_chart[i+1][j+1] / stock_price_chart[i][j]
                P[i][j] = (1 + rates[i][j] - d) / (u-d)
                # print(d)
                # print(u)
                # print(u-d)
                # print(1+rates[i][j])
                # print(1 + rates[i][j] - d)
    return P

p_vals = p_values(SP, rates)
print("p tilde")
# displayChart(p_vals)
print(p_vals)

p tilde
{0: [0.5900000000000002], 1: [0.5549999999999999, 0.6299999999999999], 2: [0.525, 0.5900000000000002, 0.6750000000000002], 3: [-1, 0.5549999999999999, 0.6299999999999999, 0.7249999999999996], 4: [1, 1, 1, 1, 1]}


In [354]:
# additive model
def exp(x,y,p=p,r=r):
    if p != -1:
        return (x*p+y*(1-p))/(1+r)
    return 0

# multiplicative model
def rnExp(x,y):
    return exp(x,y,p,r)

In [355]:
bond_payoff = list(map(g, SP[t]))
def vanillaBond(payoff): # coupon amount
    vanillaBond={}
    vanillaBond[t]=bond_payoff
    #IP=IntrinsicPut()
    #for n in range(0, N+1):
    #    PutPriceA[N].append(IP[N][n])
    for i in range(t-1, -1, -1):
        vanillaBond[i]=[]
        for j in range(i+1):
            entry = [0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(vanillaBond[i+1][j+1][0], vanillaBond[i+1][j][0], p_vals[i][j], rates[i][j]) + exp(vanillaBond[i+1][j+1][1], vanillaBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[0] = entry[0]
                entry[1] = coupon if SP[i][j] >= default_threshold or i<t else 0
            #print(entry)
            vanillaBond[i].append(entry)
    return vanillaBond

vBond = vanillaBond(bond_payoff)
print("Vanilla Bond payoff: [face value (after coupon), coupon]")
displayChart(vBond)

Vanilla Bond payoff: [face value (after coupon), coupon]


0,1,2,3,4
,,,,"[50, 2.0]"
,,,"[48.37209302325582, 2.0]","[50, 2.0]"
,,"[47.21467980526789, 2.0]","[48.82629107981221, 2.0]","[50, 2.0]"
,"[44.94132093951991, 2.0]","[43.56015817099983, 2.0]","[37.478672985782, 2.0]","[24.0, 0]"
"[39.7185738654593, 2.0]","[33.13733888553687, 2.0]","[19.739336492891, 2.0]","[0, 0]","[0, 0]"


In [356]:
def convBond(payoff):
    convBond = {}
    convBond[t] = [[max(SP[t][i], payoff[i][0]), payoff[i][1]] for i in range(t+1)]
    for i in range(t-1, -1, -1):
        convBond[i]=[]
        for j in range(i+1):
            entry = [0,0]
            if SP[i][j] != 0: 
                entry[0] = exp(convBond[i+1][j+1][0], convBond[i+1][j][0], p_vals[i][j], rates[i][j]) \
                         + exp(convBond[i+1][j+1][1], convBond[i+1][j][1], p_vals[i][j], rates[i][j])
                entry[0] = max(entry[0], SP[i][j])
                entry[1] = coupon if SP[i][j] >= default_threshold or i<t else 0
            #print(entry)
            convBond[i].append(entry)
    return convBond

cBond = convBond(bond_payoff)
print("Convertible Bond payoff: [face value (after coupon), coupon]")
print("Convertible but not callable")
displayChart(cBond)

Convertible Bond payoff: [face value (after coupon), coupon]
Convertible but not callable


0,1,2,3,4
,,,,"[70.0, 2.0]"
,,,"[61.86046511627906, 2.0]","[50.0, 2.0]"
,,"[55.723699583576945, 2.0]","[48.82629107981221, 2.0]","[50, 2.0]"
,"[49.974825597111185, 2.0]","[43.56015817099983, 2.0]","[37.478672985782, 2.0]","[24.0, 0]"
"[42.52024155223181, 2.0]","[33.13733888553687, 2.0]","[19.739336492891, 2.0]","[0, 0]","[0.0, 0]"


experiment code below

In [357]:
# x is how much to change for additive model
def simulate_one(x):
    global vol
    v = x
    
    rates = r_values(0, 0.005)
    SP = StockPrice()
    
simulate_one(1)

In [371]:
from scipy.stats import bernoulli

# rolls = {}

# for key in range(0, t+1, 1):
#         rolls[key] = [0]*(key+1)
#         for j in range(0, key+1, 1):
#             prob = max(0, min(1, p_vals[key][j]))
#             roll = bernoulli.rvs(size=1, p=prob)
#             rolls[key][j] = roll[0]


# if roll==1, then it is a head

interests = rates
for key in range(1, t+1, 1):
      for j in range(0, key+1, 1):
            prob = max(0, min(1, p_vals[key][j]))
            [roll] = bernoulli.rvs(size=1, p=prob)
            if(roll==1):
                  interests[key][j] = interests[key-1][j]+0.005
            else:
                  interests[key][j] = interests[key-1][j]-0.005


{0: [0], 1: [0, 0], 2: [0, 0, 1], 3: [0, 1, 1, 0], 4: [1, 1, 1, 1, 1]}
